## Generating Synthetic Data (CTGAN)

In [1]:
import torch
torch.cuda.current_device()

0

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
import pickle
from collections import defaultdict
from itertools import islice, combinations
from datetime import datetime as dt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt

from ctgan import CTGANSynthesizer


In [3]:
# Manage GPU Memory
import torch,gc
gc.collect()
torch.cuda.empty_cache()

In [4]:
# Control Width of Jupyter Cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
t_node_interactivity = "all"
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print('ready to run')
# Log Running Time
startTime = time.time()

ready to run


In [5]:
import os
import pickle

## Preprocess Source Data

In [9]:
# Load Raw Data
df_raw=pd.read_csv('./month/total.csv', encoding='utf-8-sig')

In [10]:
df_org_merge=df_raw[["Declaration ID", "Date", "Office ID", "Process Type", "Import Type", "Import Use", "Payment Type", 
                  "Mode of Transport", "Declarant ID", "Importer ID", "Seller ID", "Courier ID", 
                  "HS10 Code", "Country of Departure", "Country of Origin",  "Tax Rate", "Tax Type", 
                   "Country of Origin Indicator", "Net Mass", "Item Price", "Fraud", "Critical Fraud"]]

In [11]:
df_org_merge['Date']=df_org_merge['Date'].astype(str)
df_org_merge['Date']=pd.to_datetime(df_org_merge['Date'])
df_org=df_org_merge.drop_duplicates(['Date'], keep='first')
df_org=df_org.sort_values(by=['Date'], axis = 0)

In [21]:
# Sampling Train Data from Source Data
df_sample=df_org.sample(3000,replace=False)
df_sample.to_csv('./data_sample/df_sample_01.csv', index=False, encoding='utf-8-sig')

In [7]:
# Aggregate Reletive Columns
cols = ['HS10 Code', 'Country of Departure', 'Country of Origin', 'Tax Rate', 'Tax Type', 'Net Mass','Fraud', 'Critical Fraud']
df_sample['Aggregated'] =df_sample[cols].apply(lambda row: '^'.join(row.values.astype(str)), axis=1)
df_sample=df_sample.drop(cols, axis=1)
df_sample=df_sample.drop(['Item Price'], axis=1)

In [10]:
df_sample['Date']=df_sample['Date'].astype('str')

# Run CTGAN

In [11]:
categorical_columns =['Declaration ID', 'Date','Process Type','Declarant ID','Importer ID','Seller ID',
                      'Courier ID','Country of Origin Indicator','Aggregated']

In [12]:
from ctgan import CTGANSynthesizer
from ctgan import load_demo

In [13]:
ctgan = CTGANSynthesizer(verbose=True)
ctgan.fit(df_sample, categorical_columns, epochs = 100)

Epoch 1, Loss G:  5.5053,Loss D: -0.1513
Epoch 2, Loss G:  5.4834,Loss D: -0.2334
Epoch 3, Loss G:  5.6370,Loss D: -0.2905
Epoch 4, Loss G:  5.7077,Loss D: -0.2943
Epoch 5, Loss G:  5.7285,Loss D: -0.4245
Epoch 6, Loss G:  5.7067,Loss D: -0.3722
Epoch 7, Loss G:  5.8863,Loss D: -0.4257
Epoch 8, Loss G:  5.7784,Loss D: -0.5855
Epoch 9, Loss G:  6.0521,Loss D: -0.5893
Epoch 10, Loss G:  5.6965,Loss D: -0.4804
Epoch 11, Loss G:  5.8165,Loss D: -0.4722
Epoch 12, Loss G:  5.7194,Loss D: -0.4085
Epoch 13, Loss G:  5.6427,Loss D: -0.2300
Epoch 14, Loss G:  5.3994,Loss D: -0.1266
Epoch 15, Loss G:  5.1994,Loss D: -0.0384
Epoch 16, Loss G:  5.2633,Loss D: -0.0674
Epoch 17, Loss G:  4.9550,Loss D: -0.1270
Epoch 18, Loss G:  5.1929,Loss D:  0.0386
Epoch 19, Loss G:  4.8231,Loss D:  0.1363
Epoch 20, Loss G:  4.8243,Loss D:  0.0263
Epoch 21, Loss G:  4.9641,Loss D:  0.1252
Epoch 22, Loss G:  4.9302,Loss D:  0.0224
Epoch 23, Loss G:  4.6564,Loss D:  0.1212
Epoch 24, Loss G:  4.7525,Loss D:  0.1011
E

In [14]:
gc.collect()
torch.cuda.empty_cache()

In [15]:
# Create data as many as the number of sampled data
count_row = df_sample.shape[0] 
df_syn = ctgan.sample(count_row)

In [16]:
df_syn

,Declaration ID,Date,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,Seller ID,Courier ID,Country of Origin Indicator,Aggregated
0,11852334,2021-04-02,40,B,11,25,42,40,9WFVM3U,OKKWVLT,CMFKDMT,NaN,G,6506100000^DE^DE^0.0^FEU1^24.0^0^0
1,79052544,2021-03-22,30,B,11,21,11,10,BA7KAU9,5UW8U7A,CHIX0T6,5I78ZR,G,3924909000^CN^CN^0.0^FCN1^10.0^0^0
2,50324340,2020-01-29,39,B,11,21,11,41,QZE38LM,9061UCJ,52EEJFQ,MWIDNS,E,6307100000^CN^CN^10.0^A^16.0^1^1
3,22138979,2020-05-27,19,B,11,21,10,40,U8YJ1TN,1GCGR4W,WPN0LAZ,NaN,G,7404000000^AS^AS^0.0^A^1000.0^0^0
4,39051248,2020-12-26,19,B,11,21,11,10,IBG3A46,J14TF8K,ZYZ8542,NaN,B,3926909000^CN^CN^6.5^C^24.0^0^0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,94249170,2021-05-24,19,B,11,21,10,11,NJVSADX,NJ80ZX9,A99NNIP,NaN,B,6110200000^CN^CN^13.0^A^139.684^0^0
2996,62657186,2020-01-16,19,B,11,21,13,40,BAH2UUA,H0S09C1,YLYZ6JS,NaN,E,6910101000^CN^CN^8.0^A^5433.5^0^0
2997,66824991,2020-10-15,96,B,11,21,11,41,D0QBCYM,H6L122U,84ON9JR,MWIDNS,G,8528591090^CN^CN^8.0^A^64.79899999999999^0^0
2998,20709803,2021-03-16,19,B,11,21,10,11,1W60IXY,MG95UEO,UNCISOP,NaN,S,9603900000^CN^CN^2.4^FCN1^560.0^0^0


In [17]:
# Split aggregated column to original form
df_syn['HS10 Code']=df_syn["Aggregated"].str.split('^').str[0]
df_syn['Country of Departure']=df_syn["Aggregated"].str.split('^').str[1]
df_syn['Country of Origin']=df_syn["Aggregated"].str.split('^').str[2]
df_syn['Tax Rate']=df_syn["Aggregated"].str.split('^').str[3]
df_syn['Tax Type']=df_syn["Aggregated"].str.split('^').str[4]
df_syn['Net Mass']=df_syn["Aggregated"].str.split('^').str[5]
df_syn['Fraud']=df_syn["Aggregated"].str.split('^').str[6]
df_syn['Critical Fraud']=df_syn["Aggregated"].str.split('^').str[7]
df_syn = df_syn.drop(['Aggregated'],axis=1)

In [19]:
df_syn

,Declaration ID,Date,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,Seller ID,Courier ID,Country of Origin Indicator,HS10 Code,Country of Departure,Country of Origin,Tax Rate,Tax Type,Net Mass,Fraud,Critical Fraud
0,11852334,2021-04-02,40,B,11,25,42,40,9WFVM3U,OKKWVLT,CMFKDMT,NaN,G,6506100000,DE,DE,0.0,FEU1,24.0,0,0
1,79052544,2021-03-22,30,B,11,21,11,10,BA7KAU9,5UW8U7A,CHIX0T6,5I78ZR,G,3924909000,CN,CN,0.0,FCN1,10.0,0,0
2,50324340,2020-01-29,39,B,11,21,11,41,QZE38LM,9061UCJ,52EEJFQ,MWIDNS,E,6307100000,CN,CN,10.0,A,16.0,1,1
3,22138979,2020-05-27,19,B,11,21,10,40,U8YJ1TN,1GCGR4W,WPN0LAZ,NaN,G,7404000000,AS,AS,0.0,A,1000.0,0,0
4,39051248,2020-12-26,19,B,11,21,11,10,IBG3A46,J14TF8K,ZYZ8542,NaN,B,3926909000,CN,CN,6.5,C,24.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,94249170,2021-05-24,19,B,11,21,10,11,NJVSADX,NJ80ZX9,A99NNIP,NaN,B,6110200000,CN,CN,13.0,A,139.684,0,0
2996,62657186,2020-01-16,19,B,11,21,13,40,BAH2UUA,H0S09C1,YLYZ6JS,NaN,E,6910101000,CN,CN,8.0,A,5433.5,0,0
2997,66824991,2020-10-15,96,B,11,21,11,41,D0QBCYM,H6L122U,84ON9JR,MWIDNS,G,8528591090,CN,CN,8.0,A,64.79899999999999,0,0
2998,20709803,2021-03-16,19,B,11,21,10,11,1W60IXY,MG95UEO,UNCISOP,NaN,S,9603900000,CN,CN,2.4,FCN1,560.0,0,0


In [43]:
pd.options.display.float_format = '{:.2f}'.format

# Construct Item Price

In [44]:
# Data containing average unit price of each item HS code
df_price = pd.read_csv('./HS_단가적용.csv', encoding='949')

In [47]:
df_price['HS10 Code']=df_price['HS10 Code'].astype('str')
df_price['Unit Price']=df_price['Unit Price'].astype('float')

In [48]:
# Get unit price of the product from the unit price table
df_syn2=df_syn.join(df_price.set_index('HS10 Code')['Unit Price'], on='HS10 Code')
df_syn2['Net Price'] = df_syn2['Net Price'].astype('float')

In [50]:
df_syn2['Net Price'] = df_syn2['Net Price'].round(1)
df_syn2['Unit Price'] = df_syn2['Unit Price'].round(1)
df_syn2['Item Price'] = df_syn2['Net Price']* df_syn2['Unit Price']

In [54]:
pd.options.display.float_format = '{:.2f}'.format

In [57]:
df_syn2 = df_syn2.drop(['Unit Price'], axis=1)

In [58]:
df_ano=df_syn2[["Declaration ID", "Date", "Office ID", "Process Type", "Import Type", "Import Use", "Payment Type", 
              "Mode of Transport", "Declarant ID", "Importer ID", "Seller ID", "Courier ID", 
              "HS10 Code", "Country of Departure", "Country of Origin",  "Tax Rate", "Tax Type", 
               "Country of Origin Indicator", "Net Mass", "Item Price", "Fraud", "Critical Fraud"]]

# Save Generated Data

In [63]:
df_ano.to_csv('./data_syn/df_syn_ano_01.csv', index=False, encoding='utf-8-sig')

In [65]:
df_ano.shape

(3000, 22)